In [1]:
from keras.models import load_model

Using TensorFlow backend.


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import math
import copy
from PIL import Image

In [3]:
from numpy import save

In [4]:
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Flatten, Dropout, Activation, Lambda, Permute, Reshape
from keras.layers import Convolution1D, Convolution2D, ZeroPadding2D, MaxPooling2D

In [5]:
import os

In [8]:
trial_model=load_model('VGGFace.h5')

C:\Users\Om\Anaconda3\envs\opencv4\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [9]:
def features(featmodel, crpimg, transform=False): #extract face feature vector
    
    # transform=True seems more robust but I think the RGB channels are not in right order
    
    imarr = np.array(crpimg).astype(np.float32)

    if transform:
        imarr[:,:,0] -= 129.1863
        imarr[:,:,1] -= 104.7624
        imarr[:,:,2] -= 93.5940        
        aux = copy.copy(imarr)
    imarr = np.expand_dims(imarr, axis=0)
    output= featmodel.predict(imarr)[0,:]
    return output

In [10]:
def feature_extractor(inpDir,person,timestep,sample,ex_type=True): #access directory and extract face vector
    
    
    if ex_type == True:
        Y_train_labels=np.ones((5,512))
    else:
        Y_train_labels=np.zeros((5,512))  #remove after training stage is completed
        
      
    sample_space=[] #for appending different utterances
    feature_sequence = [] # for appending different timesteps
    inpDir='inpDir' #input dataset most likely to be People_cerebrus/photos
    linpDir = os.listdir(inpDir) #list all directories in dataset
    personStr= linpDir[person]
    sampleFolder = '%s\\%s' % (inpDir,personStr) #opening sample folder
    lsampleFolder = os.listdir(sampleFolder)
    for sample in lsampleFolder:
        utterFolder = '%s\\%s' % (sampleFolder,sample)
        lutterFolder = os.listdir(utterFolder)
        for utterances in lutterFolder:
            utterNumber= '%s\\%s' % (utterFolder,utterances)
            lutterNumber= os.listdir(utterNumber)
            
            frame = lutterNumber[timestep]
            i += 1
            image= "%s\\%s" % (colourFolder,frame) 
            im=Image.open(image)
            im = im.resize((224,224))
            feature_vector = features(trial_model,im, transform=True).reshape((1,1, 1, 2622))
            if i==1 :
                feature_sequence=feature_vector #done because of need of same dim. for concatenation
            else:
                feature_sequence = np.concatenate((feature_sequence,feature_vector),axis=0)

    print("Extraction completed %d"%i)
    
    WB_CNN= Sequential()
    WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation= 'relu', input_shape=(1,1,2622)))
    WB_CNN.add( Dropout(0.2) )
    WB_CNN.add(Convolution2D(1024, kernel_size=(1,1), activation='relu',input_shape=(1,1,2622)))
    WB_CNN.add( Dropout(0.2) )
    WB_CNN.add(Convolution2D(512, kernel_size=(1,1), activation='relu'))
    WB_CNN.add(Flatten())   

    WB_CNN.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    WB_CNN.fit(feature_sequence,Y_train_labels,epochs=20,batch_size=2)
    CNN_out = WB_CNN.predict(feature_sequence)
    
    print("Prediction done")
    print(CNN_out.shape)
    return CNN_out   

In [11]:
def per_to_feat(inpDir,person,sample,ex_type):
    timesteps=20
    count=0
    for i in range(timesteps):
        CNN_out=feature_extractor(inpDir,personID,wordID,i,ex_type)
        count = count + 1
        if count==1 :
            CNN_out_total= CNN_out #maintaining same dimensions
        else :
            CNN_out_total= np.concatenate((CNN_out_total,CNN_out))
        print(str(i) + 'timestep')
    
    print('Concatenation completed')
    return CNN_out_total      

In [12]:
def reshape_LSTM(sample_mat):
    feature_vector=[]
    feature_mat=[]
    count_i=0
    for i in range(10):
        count_j=0
        for j in range(0,200,10):
            j=j+i
            sample_mat_j=sample_mat[j]
            sample_mat_j=sample_mat_j.reshape((1,512))
            if count_j==0:
                feature_vector=sample_mat_j
            else:
                feature_vector=np.concatenate((feature_vector,sample_mat_j))
            count_j = count_j + 1
        
        if count_i==0:
            feature_mat=feature_vector
        else:
            feature_mat= np.concatenate((feature_mat,feature_vector))
        count_i= count_i + 1
    LSTM_input= feature_mat
    return LSTM_input

In [13]:
def BottleNeck(InpDir,person):
    p_dataset=[]
    count=0
    for i in range(5):
        count = count +1
        if personID==0 and i==0:
            CNN_out_total= per_to_feat(InpDir,person,i,True)
        else:
            CNN_out_total= per_to_feat(InpDir,person,i,False)
        if count==1 :
            p_dataset = CNN_out_total #done because of need of same dim. for concatenation
        else:
            p_dataset = np.concatenate((p_dataset,CNN_out_total),axis=0)
        print(str(i) + 'word')
    print('----------------')
    print(count)
    print(p_dataset.shape)
    return p_dataset 

def BottleNeck(InpDir,no_of_people):
    fcount=0
    for i in range(people):
        fcount= fcount+1
        p_dataset=person_extractor(i)
        p_dataset_LSTM=reshape_LSTM(p_dataset)
        if fcount==1:
            p_total_dataset=p_dataset_LSTM
        else:
            p_total_dataset=np.concatenate((p_total_dataset,p_dataset_LSTM),axis=0)
        
        print('person done')
    save('test_dataset.npy',p_total_dataset)